In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.python.platform import gfile

In [4]:
with gfile.FastGFile("SNN_294_100_sigmoid_softmax.pb",'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')


# In[3]:

sess = tf.Session()
all_ops_list = sess.graph.get_operations()

#Method 1
for ops_range in range(len(all_ops_list)):
    print([m.values() for m in all_ops_list][ops_range])
    print([m.name for m in all_ops_list][ops_range])


# In[4]:

#Method 2
with tf.Session() as sess:
    input_x = sess.graph.get_tensor_by_name("dense_5_input_2:0")
    print(input_x)

(<tf.Tensor 'dense_5_input_2:0' shape=<unknown> dtype=float32>,)
dense_5_input_2
(<tf.Tensor 'dense_5_2/kernel:0' shape=(294, 100) dtype=float32>,)
dense_5_2/kernel
(<tf.Tensor 'dense_5_2/kernel/read:0' shape=(294, 100) dtype=float32>,)
dense_5_2/kernel/read
(<tf.Tensor 'dense_5_2/bias:0' shape=(100,) dtype=float32>,)
dense_5_2/bias
(<tf.Tensor 'dense_5_2/bias/read:0' shape=(100,) dtype=float32>,)
dense_5_2/bias/read
(<tf.Tensor 'dense_5_2/MatMul:0' shape=(?, 100) dtype=float32>,)
dense_5_2/MatMul
(<tf.Tensor 'dense_5_2/BiasAdd:0' shape=(?, 100) dtype=float32>,)
dense_5_2/BiasAdd
(<tf.Tensor 'dense_5_2/Sigmoid:0' shape=(?, 100) dtype=float32>,)
dense_5_2/Sigmoid
(<tf.Tensor 'dense_6_2/kernel:0' shape=(100, 2) dtype=float32>,)
dense_6_2/kernel
(<tf.Tensor 'dense_6_2/kernel/read:0' shape=(100, 2) dtype=float32>,)
dense_6_2/kernel/read
(<tf.Tensor 'dense_6_2/bias:0' shape=(2,) dtype=float32>,)
dense_6_2/bias
(<tf.Tensor 'dense_6_2/bias/read:0' shape=(2,) dtype=float32>,)
dense_6_2/bias/re

In [7]:
import keras.backend as K
from keras.applications import VGG19
from keras.models import Model

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

In [10]:
from keras.models import load_model
model = load_model('SNN_294_100_sigmoid_softmax.h5')
export_path = 'folder_to_export'
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'images': model.input},
                                  outputs={'scores': model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: folder_to_export/saved_model.pb


In [ ]:
import argparse
import pprint

from tensorflow_serving_python.client import TFClient

pp = pprint.PrettyPrinter(indent=2)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='RPC Test.')

    parser.add_argument('--host', required=True, type=str, help='Hostname to query')
    parser.add_argument('--port', required=True, type=str, help='Port to query')
    parser.add_argument('--image', required=True, type=str, help='Image to send (JPG format)')
    args = parser.parse_args()

    data = open(args.image, "rb").read()
    client = TFClient(args.host, args.port)
    pp.pprint(client.make_prediction(data, timeout=10))